## Dataset-generator for Intercity Transportation

This notebook walks through the steps of creating the dataset of a realistic intercity public transportation network. The created data can be used to retrospectively try the electric coach scheduling software developed by Billen et al. (see link). It walks through the sequential planning steps in public transit. The user can input their desired cities in any country in the world, and the notebook will step-wise create an intercity network, charging infrastructure, line design, and timetable. Every planning step builds on top of each other, therefore, much research has already went into integrating these steps. We, however, keep the blocks modular and a little simpler. Let's get started with some configurations:

### Libraries

In [261]:
#import googlemaps
import pandas as pd
import numpy as np
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import math
import random as rand
from pathlib import Path

### Configurations:

In [ ]:
RAND_SEED = 151997
COUNTRY = 'scotland'
DATASET = 'full' # toy, small, medium, large, full
FREQUENCY = 22 # Can also be set individually for every line:

PATH_WORKSPACE = "/Users/malte/Documents/workspace/phd/eva/job_handler/"
PATH_DATA = PATH_WORKSPACE + "data/"+ COUNTRY + "/" + DATASET +"/" + str(FREQUENCY) + "F/"

# Create the path, and all missing parent folders:
Path(PATH_DATA).mkdir(parents=True, exist_ok=True)

# Planning horizon start/end date:
# E.g. 2 month planning horizon:
START_DATE = pd.to_datetime("2024-05-01 00:00:00+00:00")
END_DATE = pd.to_datetime("2024-07-01 00:00:00+00:00")

# City names that Google Maps can find: 
CITIES = {
    'Dundee, UK' : {'has_charger' : True, 'is_fixed_maintenance_location' : True}
    ,'Edinburgh, UK': {'has_charger' : False, 'is_fixed_maintenance_location' : False} 
    ,'Aberdeen, UK': {'has_charger' : True, 'is_fixed_maintenance_location' : False}
    ,'Inverness, UK': {'has_charger' : False, 'is_fixed_maintenance_location' : True}
    ,'Glasgow, UK': {'has_charger' : True, 'is_fixed_maintenance_location' : True}
    ,'Fort William, UK' : {'has_charger' : True, 'is_fixed_maintenance_location' : False}
}

# All lines:
# (origin, destination) : frequency

LINES = {
           ('Dundee, UK', 'Edinburgh, UK') : FREQUENCY
          ,('Dundee, UK', 'Aberdeen, UK') : FREQUENCY
          ,('Dundee, UK', 'Inverness, UK') : FREQUENCY
          ,('Inverness, UK', 'Aberdeen, UK') : FREQUENCY
          ,('Edinburgh, UK', 'Glasgow, UK') : FREQUENCY
          ,('Dundee, UK', 'Glasgow, UK') : FREQUENCY
          ,('Fort William, UK', 'Inverness, UK') : FREQUENCY
          ,('Dundee, UK', 'Fort William, UK') : FREQUENCY
          ,('Glasgow, UK', 'Fort William, UK') : FREQUENCY
}

# (name, range [kwh], charging [volt], charging [amps], cost)
# Fleet based on: Pelican Yutong TCe12, and GTe14.
FLEET_MAX_BATTERY_DETER = 0.07
FLEET_OPTIONS = {
    'A':{
        'range_kw_min' : 38,
        'range_kw_max': 250,
        'volts' : 600,
        'amps': 250,
        'cost': 5000,
        'kwh_per_km': 0.9
    },
    'B':{
        'range_kw_min' : 42,
        'range_kw_max': 281,
        'volts' : 600,
        'amps': 200,
        'cost': 6000,
        'kwh_per_km': 0.87
    },
    'C':{
        'range_kw_min' : 50,
        'range_kw_max': 330,
        'volts' : 600,
        'amps': 250,
        'cost': 7000,
        'kwh_per_km': 0.92
    },
    'D':{
        'range_kw_min' : 85,
        'range_kw_max': 563,
        'volts' : 600,
        'amps': 250,
        'cost': 10000,
        'kwh_per_km': 1.11
    },
    'E':{
        'range_kw_min' : 94,
        'range_kw_max': 621,
        'volts' : 600,
        'amps': 400,
        'cost': 15000,
        'kwh_per_km': 1.15
    }
}

NETWORK = nx.Graph()

### Helpers

In [263]:
class Helpers:
      
    def roundup(x, target):
        return math.ceil(x / float(target)) * int(target)
    
    def rounddown(x, target):
        return math.floor(x / float(target)) * int(target)

Use the Google Maps API to find the distances between all cities. The code can compute the distance matrix, but if available, it will use previously computed distance.csv.

In [264]:
# MAPS_API_KEY = "INSERT_API_KEY_HERE"
# GMAPS = googlemaps.Client(key=MAPS_API_KEY)
DISTANCES = pd.read_csv(PATH_WORKSPACE + "data/" + COUNTRY + "/distances.csv").set_index(["from","to"])

# Create the distance matrix using the Google Maps API:
# DISTANCES = pd.DataFrame(columns=["from","to","duration","distance"]).set_index(["from","to"])
# for origin in CITIES:
#    for destination in CITIES:
#        if origin != destination:
#            response = GMAPS.distance_matrix(origin, destination)
#            distance = response['rows'][0]['elements'][0]['distance']['value'] / 1000 # response in cm
#            duration = Helpers.roundup(int(response['rows'][0]['elements'][0]['duration']['value'] * 1.1), 60) # response in seconds. round to nearest minute. Added additional 10% to travel time, bus slower and has stops along the way.

#            DISTANCES.loc[(origin, destination),:] = [duration, distance]
#        else:
#            DISTANCES.loc[(origin, destination),:] = [np.nan, np.nan]

# DISTANCES.to_csv("distances.csv")

# Compute the discharged battery amount
# High average km/h discharges the battery faster. Therefore, we estimate a slightly higher battery drainage on routes with higher km/h.
# DISTANCES['kmh'] = DISTANCES['distance'].div((DISTANCES['duration'] / 3600.0))
# DISTANCES['discharge'] = DISTANCES['distance'] * (0.9 * (DISTANCES['kmh'] / DISTANCES['kmh'].median())**0.5)

# DISTANCES = DISTANCES.fillna(value=0.0)

### Datasets:
<b><i>locations.csv:</b> Id, Type, Name, [longitude, latitude]</i><br>
<b><i>location_distances.csv:</b> FromLocationId,ToLocationId,Duration,Distance </i><br>
<b><i>vehicles.csv:</b> Id,BatteryMinKWh,BatteryMaxKWh,InitialChargerID,InitialStartTime,InitialSOC,ChargingSpeedVolts,ChargingSpeedAmps,NumberPlate,OdometerReading,OdometerLastMaintenance,InRotation,Kwh_per_km </i><br>
<b><i>chargers.csv:</b> Id,LocationId,Capacity,ChargingSpeedVolts,ChargingSpeedAmps </i><br>
<b><i>chargers_unavailabilities.csv:</b> Id,ChargerId,StartTime,EndTime,CapacityReduction </i><br>
<b><i>vehicle_activities.csv:</b> Id,VehicleId,StartTime,EndTime,StartLocationID,EndLocationID </i><br>
<b><i>trips.csv:</b> Id,StartTime,EndTime,StartLocationID,EndLocationID</i>

### Network Design:

1) Create the line network with cities as nodes and lines as edges. 
2) Find the maintenance locations by searching for a minimum node cover on the line network. This ensures not too many maintenance locations are created but they are spread evenly across the map. The minimum node cover is obtained by solving the following simple integer program using the Gurobi solver:
$
\begin{align}
&\min& \sum_{v \in V} x_{v}& \\
&s.t.& x_i + x_j \geq 1\ &\forall (i,j) \in E \\
&& x_v \in \{0,1\}\ &\forall v \in V
\end{align}
$
The problem is NP-complete, however, for small instances, an optimal soluition can be obtained by commercial solvers. If the network size is larger, an optimal solution for the vertex cover might not be obtained or may take a while to solve.

3. Finally, create the 'locations.csv' file. Every city has one stop, and can have one charger if attribute 'has_charger' is true. If it is part of the minimum vertex cover, it also has a maintenance station to maintain the vehicles.

In [265]:
# Create locations.csv:
# Initialise network design:
NETWORK.add_nodes_from(CITIES)
NETWORK.add_edges_from(LINES)

#####
# Compute a minimum vertex cover in the line network:
m_sn = gp.Model("VertexCover")
m_sn.setParam("OutputFlag", 0)

# Variables:
x = m_sn.addVars(NETWORK.nodes, vtype=GRB.BINARY, obj=1.0)

# Constraints:
m_sn.addConstrs((x[source] + x[sink] >= 1 for source, sink in NETWORK.edges), name='c')

for node in NETWORK.nodes:
    if(CITIES[node]['is_fixed_maintenance_location']):
        m_sn.addConstr(x[node] == 1)

# Optimise:
m_sn.optimize()
#####

SERVICE_CITIES = {node for node in NETWORK.nodes if int(x[node].X) == 1}

# Prepare locations dataframe:
df_locations = pd.DataFrame(columns=['Id', 'Type', 'Name', 'Address']).set_index('Id')

for city, attr in CITIES.items():
    df_locations.loc[len(df_locations)] = ["Stop", "Stop @ " + city[:3].upper(), city] # Every city has one terminal bus stop of a line

    # Check if the city has a charging station:
    if attr['has_charger']:
        df_locations.loc[len(df_locations)] = ["Charger", "Charger @ " + city[:3].upper(), city]

    if city in SERVICE_CITIES: # Every city in the minimum vertex cover has a maintenance location
        df_locations.loc[len(df_locations)] = ["Maintenance", "Maintenance @ " + city[:3].upper(), city]

# Store the 'locations.csv' data frame:
df_locations[['Type', 'Name']].to_csv(PATH_DATA + "locations.csv")

In [266]:
# Create the locations_distances.csv file:
df_locations_distances = pd.DataFrame(columns=['FromLocationsId', 'ToLocationId', 'Duration', 'Distance']).set_index(['FromLocationsId', 'ToLocationId'])

for fromId, fromRow in df_locations.iterrows():
    for toId, toRow in df_locations.iterrows():
        # Create every location pairing:
        if fromId == toId:
            df_locations_distances.loc[(fromId, toId), :] = [0,0]
        else:
            df_locations_distances.loc[(fromId, toId), :] = [np.nan, np.nan]

        if fromRow['Address'] != toRow['Address']:
            # Inter-city routing:
            
            if fromRow['Type'] == 'Stop' and toRow['Type'] == 'Stop':
                # Only route from 'Stop' -> 'Stop'
                if (fromRow['Address'], toRow['Address']) in LINES or (toRow['Address'], fromRow['Address']) in LINES:
                    dist = DISTANCES.loc[(fromRow['Address'], toRow['Address']),:] # ["duration", "distance"]
                    df_locations_distances.loc[(fromId, toId), :] = [int(dist['duration']), int(dist['distance'])]
        else:
            # Intra-city routing:
           
            if fromRow['Type'] == 'Stop' and toRow['Type'] == 'Charger' or fromRow['Type'] == 'Charger' and toRow['Type'] == 'Stop':
                # 'Stop' <-> 'Charger'
                df_locations_distances.loc[(fromId, toId), :] = [0, 0] # The charger is assumed to always be at the terminal stop.
                
            elif fromRow['Type'] == 'Stop' and toRow['Type'] == 'Maintenance' or fromRow['Type'] == 'Maintenance' and toRow['Type'] == 'Stop':
                # 'Stop' <-> 'Maintenance'
                distance = 5
                duration = 8 * 60
                df_locations_distances.loc[(fromId, toId), :] = [duration, distance] # The charger is assumed to always be at the terminal stop.

                
            elif fromRow['Type'] == 'Charger' and toRow['Type'] == 'Maintenance' or fromRow['Type'] == 'Maintenance' and toRow['Type'] == 'Charger':
                # 'Charger' <-> 'Maintenance'
                distance = 5
                duration = 8 * 60
                
                df_locations_distances.loc[(fromId, toId), :] = [duration, distance] # The charger is assumed to always be at the terminal stop.
            else:
                pass
            
df_locations_distances.to_csv(PATH_DATA + "locations_distances.csv")

In [267]:
# Create the chargers.csv file:
# Use the locations.csv file now, to obtain the correct location id:

df_chargers = pd.DataFrame(columns=['Id','LocationId','Capacity','ChargingSpeedVolts','ChargingSpeedAmps']).set_index(['Id'])

def recharging_dur(v, d) -> int:
    return ((d * (FLEET_OPTIONS[v]['kwh_per_km'] + FLEET_MAX_BATTERY_DETER))) / (float(FLEET_OPTIONS[v]['volts'] * FLEET_OPTIONS[v]['amps'] / (1000 * 60)))

volts = 1000 # Fast-charger config only
amps = 400 # Fast-charger config only
buffer_maintenance = 25
buffer_pull_in_out = 10

for locId, row in df_locations.iterrows():
    if row['Type'] == 'Charger':
        capacity = 1.0

        for (fromCity, toCity), frequency in LINES.items():
            if fromCity == row['Address'] or toCity == row['Address']:
                headway = (1440 * 0.75) / (frequency * 0.875 ) # every headway minutes there is an outbound trip with potential overlapping charging session.

                dist_out = DISTANCES.loc[(fromCity, toCity),'distance']
                dist_ret = DISTANCES.loc[(toCity, fromCity),'distance']
                ref_recharge = 0
                for v in FLEET_OPTIONS.keys():
                    ref_recharge = max(ref_recharge, recharging_dur(v, dist_ret + dist_out + buffer_maintenance))
                
                capacity += ref_recharge / headway
        
        capacity = math.ceil(capacity)
        df_chargers.loc[len(df_chargers)] = [locId, math.ceil(capacity), volts, amps]

df_chargers.to_csv(PATH_DATA + "chargers.csv")

In [268]:
# Create the chargers_unavailabilities.csv file:
# This can be filled if desired to stress-test the allocation if certain chargers are limited in availability for some period of times.

df_chargers_unavailability = pd.DataFrame(columns=['Id','ChargerId','StartTime','EndTime','CapacityReduction']).set_index(['Id'])

# ...

df_chargers_unavailability.to_csv(PATH_DATA + "chargers_unavailability.csv")


In [269]:
network_directed = NETWORK.to_directed()
network_lines = nx.DiGraph()
network_lines.add_nodes_from(network_directed.edges)
range_extra_buffer = 0.05
max_distance = 0

for (o_l,d_l) in network_lines.nodes:
    for (o_r,d_r) in network_lines.nodes:
        if d_l == o_r:
            network_lines.add_edge((o_l,d_l),(o_r,d_r))

for name, attr in FLEET_OPTIONS.items():
    range_v = math.floor((attr['range_kw_max'] - attr['range_kw_min']) * (1 - range_extra_buffer))
    range_dist = math.floor(range_v / float((attr['kwh_per_km'] + FLEET_MAX_BATTERY_DETER)))
    max_distance = max_distance if max_distance > range_dist else range_dist

all_routes = list()

for oc in CITIES:
    if CITIES[oc]['has_charger']:
        for dc in CITIES:
            if CITIES[dc]['has_charger']:
                source_nodes = [(o,d) for (o,d) in network_lines.nodes if o == oc]
                target_nodes = [(o,d) for (o,d) in network_lines.nodes if d == dc]

                for s in source_nodes:
                    for t in target_nodes:
                        paths = list(nx.all_simple_paths(G=network_lines,source=s,target=t,cutoff=6))
                        for path in paths:
                            dist_path = 0
                            dur_path = 0
                            for (o,d) in path:
                                dist_path += DISTANCES.loc[(o, d),'distance'] 
                                dur_path += DISTANCES.loc[(o, d),'duration']

                            if dist_path < max_distance:
                                all_routes.append((path, dist_path, dur_path, (oc,dc)))

all_line_routes = list()
for (route, dist, dur, (oc,dc)) in all_routes:
    line_set = set()
    for (o,d) in route:
        if (o,d) in LINES:
            line_set.add((o,d))
        if (d,o) in LINES:
            line_set.add((d,o))

    # check if this set already exist:
    found = False
    replace = len(all_line_routes)
    idx = 0
    for (cRoute, cDist,_,_) in all_line_routes:
        if cRoute == line_set:
            found = True
            if dist > cDist or oc != dc:
                replace = idx
                dist = max(dist, cDist)

        idx += 1

    if not found:
        all_line_routes.append((line_set, dist,dur,(oc,dc)))
    else:
        if replace < len(all_line_routes):
            del all_line_routes[replace]
            all_line_routes.append((line_set, dist,dur,(oc,dc)))

all_line_routes_idx = [l for l in range(len(all_line_routes))]

### 
m_vr = gp.Model("VehicleRouting")
m_vr.setParam("OutputFlag", 0)

# Variables:
x = list()
for idx in all_line_routes_idx:
    x.append(m_vr.addVar(vtype=GRB.BINARY,obj=(all_line_routes[idx][1]*all_line_routes[idx][1]))) # Sqaure the distance to prefer shorter routes.

# Constraints:
for l in LINES:
    m_vr.addConstr(sum(x[idx] for idx in all_line_routes_idx if l in all_line_routes[idx][0]) == 1)

# Optimise:
m_vr.optimize()
#####

line_routing_result = [all_line_routes[idx] for idx in all_line_routes_idx if int(x[idx].X) == 1]
line_routing_result

[({('Dundee, UK', 'Edinburgh, UK')},
  np.float64(192.293),
  np.float64(11400.0),
  ('Dundee, UK', 'Dundee, UK'))]

In [270]:
# Create the vehicles.csv file:
rand.seed(RAND_SEED)

df_vehicles = pd.DataFrame(columns=['Id','BatteryMinKWh','BatteryMaxKWh','InitialChargerID','InitialStartTime','InitialSOC','ChargingSpeedVolts','ChargingSpeedAmps','NumberPlate','OdometerReading','OdometerLastMaintenance','InRotation','Cost','kwh_per_km']).set_index('Id')

# Set the lower bound risk [%] to reserve as a buffer. This counters elasticity in battery discharge on all routes. 
range_extra_buffer = 0.05
time_buffer_charging = 10 * 60
time_buffer_turnaround = 30 * 60 + ((24*60*60)/FREQUENCY)/2 # If the timetable is not perfectly synchronised, then the vehicle might have to wait on the route.
extra_spare_vehicle = 0.2 # Account for an additional 10% of spare vehicles at every line. This ensures if for instance the estimated amount is close to the ceiling, a spare vehicle is added.
odomoter_distance_lb = 10000
odometer_distance_ub = 300000
odometer_last_maintenance_lb = 5
odometer_last_maintenance_ub = 1000

def get_extra_discharge_coeff(odo):
    assert odo <= odometer_distance_ub and odo >= odomoter_distance_lb
    return round(float(odo - odomoter_distance_lb) / float(odometer_distance_ub - odomoter_distance_lb) * FLEET_MAX_BATTERY_DETER, 2)

numberPlateCounter = {vType : 0 for vType in FLEET_OPTIONS.keys()}

# Vehicles only start at chargers.
# Every line must have sufficiently many vehicles available at either end-point.
# Compute the full cycle: Travel time + Recharging time + buffer to estimate number of required vehicles for every line. 
# Then distribute the vehicle evenly at the chargers of both end-points of the line.

# USE line_routing_result to determine the fleet options:
for (line_set, dist, dur, (oc,dc)) in line_routing_result:
    frequency = 0
    total_duration = dur
    charger_options = list()
    for (o,d) in line_set:
        frequency = max(frequency, LINES[(o,d)])
        total_duration += time_buffer_turnaround
        if CITIES[o]['has_charger']:
            charger_options.append(o)
        if CITIES[d]['has_charger']:
            charger_options.append(d)
    
    if oc != dc:
        total_duration *= 2 # Need to make return trip as well

    vehicleType = ''
    vehicleCost = math.inf
    vehicleAmount = 0

    for name, attr in FLEET_OPTIONS.items():
        total_range = math.floor((attr['range_kw_max'] - attr['range_kw_min']) * (1 - range_extra_buffer))
        range_dist = math.floor(total_range / float((attr['kwh_per_km'] + FLEET_MAX_BATTERY_DETER)))

        if oc != dc:
            vehicle_recharging_dur = 2 * (dist * ((1000.0 * 3600.0) / (attr['volts'] * attr['amps'])) + time_buffer_charging)
        else:
            vehicle_recharging_dur = dist * ((1000.0 * 3600.0) / (attr['volts'] * attr['amps'])) + time_buffer_charging

        if range_dist > dist:
            # Vehicle eligible to serve on the line:
            amount = math.ceil(((1 + extra_spare_vehicle) * frequency * (total_duration + vehicle_recharging_dur)) / 86400.0) 
            totalCost = amount * attr['cost']

            vehicleType, vehicleCost, vehicleAmount = (name, totalCost, amount)  if totalCost < vehicleCost else (vehicleType, vehicleCost, vehicleAmount)
        
    if len(vehicleType) > 0:
        # Distribute the required number of vehicles:
        attr = FLEET_OPTIONS[vehicleType]
        v = 0

        while v < vehicleAmount:
            odometer_reading = rand.randint(odomoter_distance_lb, odometer_distance_ub) # Sample the odometer reading of the vehicle.
            odometer_last_maintenance = odometer_reading - rand.randint(odometer_last_maintenance_lb, odometer_last_maintenance_ub)  # Sample the odometer reading the last time it was served:
            extra_discharge = get_extra_discharge_coeff(odometer_reading)

            # Alternate between the two charger:
            address = charger_options[v % len(charger_options)]
            chargerId = df_chargers[df_chargers['LocationId'] == df_locations.loc[(df_locations['Type'] == 'Charger') & (df_locations['Address'] == address)].index.values[0]].index.values[0]

            df_vehicles.loc[len(df_vehicles)] = [
                attr['range_kw_min'],
                attr['range_kw_max'],
                chargerId,
                START_DATE,
                attr['range_kw_min'],
                attr['volts'],
                attr['amps'],
                str(vehicleType) + str(numberPlateCounter[vehicleType]),
                odometer_reading,
                odometer_last_maintenance,
                0,
                attr['cost'],
                round(attr['kwh_per_km'] + extra_discharge,2)
            ]

            numberPlateCounter[vehicleType] += 1
            v += 1
    else:
        # Output a warning, that there is no suitable vehicle to cover the line:
        print("Warning: ", "No eligible vehicle exist in the fleet options to cover the line (", line_set, ").")
        assert True

df_vehicles.to_csv(PATH_DATA + "vehicles.csv")

In [271]:
# Create the maintenances.csv file:
rand.seed(RAND_SEED)

df_maintenances = pd.DataFrame(columns=['Id','StartTime','EndTime','MaintenanceLocationID','VehicleId']).set_index('Id')
working_hours_start, working_hours_end = 6, 22 # Maintenances can only be done within working hours. Based on two-8-hour shift model: morning shift from 6 - 14, and afternoon from 14 - 22.

for maintenanceLocId, row in df_locations.loc[df_locations['Type'] == 'Maintenance'].iterrows():
    # Find the number of incoming/outgoing lines, and compute the number of maintenance per location:
    maintenance_duration = rand.choice([120, 140, 160]) # Random duration in minutes. 
    amount = math.floor(len(df_vehicles) * len(NETWORK[row['Address']]) / sum(len(NETWORK[city]) for city in SERVICE_CITIES))
    headway = Helpers.rounddown((((working_hours_end - working_hours_start) * 60) - maintenance_duration) / amount, 1)
    
    # Start at the beginning of the planning horizon:
    maintenance_start_time = START_DATE.replace(hour=working_hours_start) - pd.Timedelta(days=1)

    # Stop whenever the end of the planning horizon is reached:
    while maintenance_start_time < END_DATE:
        # Reset the day to the beginning of the working day:
        maintenance_start_time = maintenance_start_time.replace(hour=working_hours_start, minute=0, second=0)
        
        # Create the maintenance at the maintenance location:
        i = 0
        while i < amount:
            # Add the maintenance :
            df_maintenances.loc[len(df_maintenances)] = [maintenance_start_time, maintenance_start_time + pd.Timedelta(minutes=maintenance_duration), maintenanceLocId, ""]
            
            # Increment the maintenance start time:
            maintenance_start_time += pd.Timedelta(minutes=headway)

            i += 1
            
        # Move on to the next day:
        maintenance_start_time += pd.Timedelta(days=1)

df_maintenances.to_csv(PATH_DATA + "maintenances.csv")

The timetable and trips are generated using the line frequencies. All trips take place periodically, and there is no difference between weekend and weekdays. However, trips take place overnight on a reduced sevice. In this regard, we differentiate between nighttime-trips (12am - 6am) and daytime-trips (6am - 12am). During the nighttime, the frequency of trips is reduced by around a half. Some deviations can occur due to rounding errors. 

With a focus on the passenger experience, the timetable is subsequentially optimised to minimise transfer times between lines. While many passengers may only travel one line, some passengers might want to transfer to reach destinations further away. To ensure that the passenger has sufficient time to transfer, we account for a buffer at the end of a trip. 

Given departure $T^D$, and arrival times $T^A$ for all trips of a line $i \in I_{\ell}$ on one day, and their respective start- and end-stop, the following minimisation problem can be formulated to minimise the transfer times at all stations:

$$
\begin{align}
&\min& \min \sum_{i,j \in K} t_{ij} && \\
&s.t.&  (T_j^D + h_{l(j)}) - (T_i^A + h_{l(i)} + \Delta^{B}) + z_{ij} * T^{P} &= t_{ij}  \ &\forall (i,j) \in H \\
&& 0 &\leq h_{l} \leq (T^{P} - 1) - \max_{i \in I_{\ell}}\{T_i^D\}  &\forall \ell \in L \\
&& 0 &\leq t_{ij} &\forall (i,j) \in H \\
&& z_{ij} &\in \{0,1\}\ &\forall (i,j) \in H  
\end{align}
$$

The set $K$ considers all trip-pairings such that a transfer takes place at the same terminal stop and may be possible on the same day. Then, $t_{ij}$ describes the transfer between two trips, $i,j \in I$. Maintaining the line's headyway, only one variable exists per line $h_{\ell}$ to shift all departure times of line $\ell$. Variable $z_{ij}$ is a binary indicator, if the connection can only be made on the next day. The optimisation problem finds timetable shifts for all lines such that the 

In [272]:
def get_var_objective_value(ls, le, rs, re) -> float:
    for route, _, _, (o,d)  in line_routing_result:
        # Get all transfer cities:
        transfer_cities = set()
        for (oc,dc) in route:
            if oc != o and oc !=d:
                transfer_cities.add(oc)
            elif dc != o and dc !=d:
                transfer_cities.add(dc)

        # Check if the two lines are represented in the route:
        flag_l = False
        flag_r = False
        for (oc,dc) in route:
            if (ls == oc and le == dc) or (le == oc and ls == dc):
                flag_l = True
            if (rs == oc and re == dc) or (re == oc and rs == dc):
                flag_r = True

        # Check if both lines are in the route:
        if flag_l and flag_r:
            # Check if the city is a transfer city:
            if le == rs:
                if le in transfer_cities:
                    return 1.0
                else:
                    return 0.0
    
    return 0.0

In [273]:
# Create the trips.csv file:
df_trips = pd.DataFrame(columns=['Id','StartTime','EndTime','StartLocationID','EndLocationID']).set_index('Id') 

daytime_start, daytime_end = 6, 24      # 0600 - 2400: Daytime Timetable
nighttime_start, nighttime_end = 0, 6   # 0000 - 0600: Nightime Timetable (reduced service: instead of 25% of line frequency, only 12.5%)
nighttime_frequency = 0.125 # Reduced service by 50%. Why? Because the night time represents a quarter of the day, but only 12.5% of the expected trips during the day (instead of 25%).
bufferEnd = 15 # Plan for a buffer of 15 minutes at the end of a trip to allow a feasible transfer. 
line_id = 0

# Create an initial timetable (just one day with minutes starting from daytime_start)
df_timetable = pd.DataFrame(columns=['Id','StartTime','EndTime','StartLocationID','EndLocationID', 'LineId']).set_index('Id') 

# Iterate over every line:
for (fromCity, toCity), frequency in LINES.items():
    frequency_night = math.ceil(nighttime_frequency * frequency)
    frequency_day = frequency - frequency_night

    headway_night = math.ceil((((nighttime_end - nighttime_start) * 60)) / (frequency_night))
    headway_day = math.ceil((((daytime_end - daytime_start) * 60)) / (frequency_day))
    
    trip_start_time = 0 # START_DATE.replace(hour=daytime_start) - pd.Timedelta(days=1)

    distOutbound = DISTANCES.loc[(fromCity, toCity),:]  # ["duration", "distance", "discharge"]]
    distReturn = DISTANCES.loc[(toCity, fromCity),:]    # ["duration", "distance", "discharge"]]
    fromStop = df_locations.loc[(df_locations['Type'] == 'Stop') & (df_locations['Address'] == fromCity)].index.values[0]
    toStop = df_locations.loc[(df_locations['Type'] == 'Stop') & (df_locations['Address'] == toCity)].index.values[0]

    # Reset the day to the beginning of the working day:
    # trip_start_time = trip_start_time.replace(hour=daytime_start, minute=0, second=0)

    i = 0
    while i < frequency:
        # Add the trip:
        # df_trips.loc[len(df_trips)] = [trip_start_time, trip_start_time + pd.Timedelta(seconds=distOutbound['duration']), fromStop, toStop]
        # df_trips.loc[len(df_trips)] = [trip_start_time, trip_start_time + pd.Timedelta(seconds=distReturn['duration']), toStop, fromStop]
        df_timetable.loc[len(df_timetable)] = [int(trip_start_time), int(trip_start_time + distOutbound['duration'] / 60), int(fromStop), int(toStop), line_id]
        df_timetable.loc[len(df_timetable)] = [int(trip_start_time), int(trip_start_time + distReturn['duration'] / 60), int(toStop), int(fromStop), line_id+1]
               
        # Increment the trip start time:
        # If day-time, use daytime-headway. Otherwise, switch to nighttime:
        if i < frequency_day:
            # trip_start_time += pd.Timedelta(minutes=headway_day)
            trip_start_time += headway_day
        else:
            # trip_start_time += pd.Timedelta(minutes=headway_night)  
            trip_start_time += headway_night

        i +=1
    
    # Increment the line id:
    line_id += 2

# ------------------------------------------------------
# Optimise the timetable, s.t. transfer-times at cities are minimised.
# However, the headway must always remain the same, hence only one decision can be made per line.
mTT = gp.Model("TimetableOpt")

# Vars:
h = mTT.addVars(df_timetable['LineId'].unique(), lb=0.0, ub = (1440 - df_timetable.groupby('LineId').max()['StartTime'] - 1), vtype = GRB.INTEGER)

mTT.update()

# Constraints:
for i, row_i in df_timetable.iterrows(): 
    for j, row_j in df_timetable.iterrows():
        if row_i['EndLocationID'] == row_j['StartLocationID']:
            if (row_j['StartTime'] + h[row_j['LineId']].UB) - (row_i['EndTime'] + h[row_i['LineId']].LB + bufferEnd) >= 0:
                obj_value = get_var_objective_value(
                    df_locations[df_locations.index == row_i['StartLocationID']]['Address'].values[0], 
                    df_locations[df_locations.index == row_i['EndLocationID']]['Address'].values[0], 
                    df_locations[df_locations.index == row_j['StartLocationID']]['Address'].values[0], 
                    df_locations[df_locations.index == row_j['EndLocationID']]['Address'].values[0])
                
                t = mTT.addVar(obj=obj_value, lb=0.0)
                z = mTT.addVar(vtype=GRB.BINARY) 

                mTT.addConstr((row_j['StartTime'] + h[row_j['LineId']]) - (row_i['EndTime'] + h[row_i['LineId']] + bufferEnd) + z * (24 * 60) == t)

# Optimise
mTT.optimize()

# Store the updated headways for every line:
df_timetable.loc[:, 'StartTime'] = df_timetable.apply(lambda r : int(r['StartTime'] + h[r['LineId']].X), axis=1)
df_timetable.loc[:, 'EndTime'] = df_timetable.apply(lambda r : int(r['EndTime'] + h[r['LineId']].X), axis=1)
# ------------------------------------------------------

# Given the timetable, create the trips:
trip_start_time = START_DATE.replace(hour=daytime_start) - pd.Timedelta(days=1)
while trip_start_time < END_DATE:
    trip_start_time = trip_start_time.replace(hour=daytime_start, minute=0, second=0)

    df_timetable_cur = df_timetable[['StartTime','EndTime','StartLocationID','EndLocationID']]

    df_timetable_cur['StartTimeDate'] = df_timetable_cur['StartTime'].apply(lambda m : trip_start_time + pd.Timedelta(minutes=m))
    df_timetable_cur['EndTimeDate'] = df_timetable_cur['EndTime'].apply(lambda m : trip_start_time + pd.Timedelta(minutes=m))

    df_trips = pd.concat([df_trips, df_timetable_cur], ignore_index=True)

    trip_start_time += pd.Timedelta(days=1)

df_trips = df_trips.drop(columns=['StartTime', 'EndTime'])
df_trips = df_trips.rename(columns={
    'StartTimeDate': 'StartTime',
    'EndTimeDate': 'EndTime'
})
df_trips = df_trips[['StartTime','EndTime','StartLocationID','EndLocationID']]
df_trips['Id'] = df_trips.index
df_trips = df_trips.set_index('Id')
df_trips.to_csv(PATH_DATA + 'trips.csv')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 110 rows, 222 columns and 440 nonzeros
Model fingerprint: 0xc4bde3bf
Variable types: 110 continuous, 112 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [2e+01, 1e+03]
Presolve removed 106 rows and 217 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 8 nonzeros
Variable types: 0 continuous, 5 integer (4 binary)
Found heuristic solution: objective 22680.000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 14 (of 14 available processors)

Solution count 1: 22680 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.268000000000e+04, best bound 2.268000000000e+04, gap 0.0000%
